In [1]:
import pandas as pd
import numpy as np
import re

from io import StringIO

from sklearn.externals import joblib
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.datasets import make_regression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer


In [3]:
###читаем даатасет c фичами фильмов
df_f = pd.read_csv('./movie_titles.csv', names=['MovieID', 'Year', 'Title'])
df_f = df_f.set_index('MovieID')

###читаем даатасет с инфой о пользователях
f = open('./combined_data_1.txt')
file = f.read()

movieID_u = list(map(lambda x: int(x[:-1]), re.findall(r'\d+\:', file)))
df_u = re.split(r'\d+\:', file)
df_u = df_u[1:]    

df = []
for df_i, movieID_i in zip(df_u, movieID_u):
    sub_df = pd.read_csv(StringIO(df_i), names=['UserID', 'Score', 'Date'])
    sub_df['MovieID'] = movieID_i
    df.append(sub_df)


df = pd.concat(df)
df_f = df_f.drop(df_f[df_f.index>df.MovieID.unique().shape[0]].index)
#print(df_f)
#print(df)


In [4]:
u_size = df.UserID.unique().shape[0]
m_size = df.MovieID.unique().shape[0]
#y_size = df_f.Year.unique().shape[0]
#s_size = df.Date.unique().shape[0]

In [22]:
df.sort_values(by=['UserID', 'MovieID'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)

           UserID  Score        Date  MovieID
0               6      3  2004-09-15       30
1               6      3  2004-09-15      157
2               6      4  2004-09-15      173
3               6      5  2004-10-10      175
4               6      2  2004-11-24      191
5               6      3  2004-09-22      197
6               6      3  2005-11-25      241
7               6      4  2004-09-27      295
8               6      3  2005-04-20      299
9               6      4  2004-09-15      329
10              6      3  2005-05-15      361
11              6      3  2004-10-16      445
12              6      5  2004-09-15      457
13              6      3  2005-11-25      468
14              6      3  2004-11-17      494
15              6      3  2005-12-04      501
16              6      4  2005-10-26      528
17              6      4  2004-09-27      564
18              6      3  2005-01-20      580
19              6      3  2005-12-04      658
20              6      3  2004-03-

In [13]:
### разреженная матрица
sm = lil_matrix((df.shape[0], u_size + m_size))
#print(sm.shape)

In [7]:
j=0
temp = df.UserID[0]
#print (df[df.UserID==6].shape)

In [31]:
for i in range(df.shape[0]):
    if(temp<df.UserID[i]):
        j+=1
        temp = df.UserID[i]
    #print (j)
    sm[i, j] = 1    
    sm[i, u_size+df.MovieID[i]-1] = 1   
    if i % 50000 == 0:
        print("\r Progress: {}/{} ({}%)".format(i, df.shape[0], int(((i+1) / df.shape[0]) * 100)), end="")
    #print(i, j, sm[i,j])
    #print(i, j1, sm[i,j1])
print(sm[0,0])

H:\programs\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


 Progress: 0/24053764 (0%)

KeyboardInterrupt: 

In [42]:
sm = csr_matrix(sm)
joblib.dump(sm, 'sparse_df_1.bin')

['sparse_df_1.bin']

In [32]:
sm = joblib.load('sparse_df_1.bin')

In [48]:
print(sm[0, :])

  (0, 0)	1.0
  (0, 785)	1.0
  (0, 378549)	1.0
  (0, 470787)	1.0


По лемме 1 нелинейную часть можно представить в виде: $\sum_{i=1}^n \sum_{j=i+1}^n \langle {v}_i, {v}_f\rangle x_ix_j = \frac{1}{2}\sum_{f=1}^k \left( \left(\sum_{i=1}^n v_{i,f}x_i \right)^2 - \sum_{i=1}^n v^2_{i,f}x^2_i \right)$
<br>
Производная $\frac{dy}{dv}$ нелинейной части бдует следующей:
<br>
(1) = $\left( \left( \sum_{i=1}^n v_{i,f}x_i \right)^2 \right)' _{v_if} = 2\sum_{i=1}^n \left(v_{i,f}x_i \sum_{k=1}^n x_k\right)$ или в матричном виде $\sum_{f=1}^k2 (X\mathbb{1})^TXV_f$
<br>
(2) = $ \left( \sum_{i=1}^n v_{i,f}^2x_i^2 \right)' _{v_if} = 2\sum_{i=1}^n v_{i,f}x_i^2 $ или в матричном виде $\sum_{f=1}^k2 X^TXV_f$
<br>
<br>
Тогда
<br>
$\frac{dy}{dv_f} = \frac{1}{2}\sum_{f=1}^k \left( (1) - (2) \right)$ или в матричном виде $\sum_{f=1}^k(X\mathbb{1}-X)^TXV_f$
<br>
<br>
И тогда $\frac{dy}{dv}$ в матричном виде $(X\mathbb{1}-X)^TXV\mathbb{1}$

In [49]:
def narrTheta(lr, iter_num):
	return lr/np.sqrt(iter_num)

def sgdFFM(X, y, max_iter = 10000, step = 1e-5, batch_size = 64, k = 8):
	w0 = 0
	iter_num = 1
	N = X.shape[0]
	np.random.seed(42)
	w1 = np.random.normal(size=X.shape[1])


	while iter_num <= max_iter:
		random_batch = np.random.choice(N, batch_size)
		new_y = predict(X[random_batch], w0, w1)
		print(X[random_batch].shape)

		dy = 2*(new_y - y[random_batch])/N
		#print(iter_num, dy)
		w0 -= dy.mean()*step
		w1 -= (X[random_batch].T@dy).mean() * narrTheta(step, iter_num)
		iter_num += 1
	return w0, w1

def predict(X, w0, w1):
	return w0 + X@w1

In [50]:
df= pd.DataFrame()
df['A'] = np.random.randint(0,5000, size=40000)
df['B'] = np.random.randint(0,50, size=40000)
df['C'] = np.random.randint(0,60, size=40000)
df['D'] = np.random.randint(0,75, size=40000)
df['AA'] = np.random.randint(0,5, size=40000)
df['BB'] = np.random.randint(0,15, size=40000)
df['CC'] = np.random.randint(0,3785, size=40000)
df['DD'] = np.random.randint(0,95, size=40000)
df['EE'] = np.random.randint(0,8, size=40000)
df['E'] = np.random.randint(0,115, size=40000)
df['Target'] = df['B'] + df['AA']+ df['BB'] + df['CC'] + df['A']

In [52]:
folds_index = 5
fold_size = round(df.shape[0] / folds_index)
RMSE_test = []
RMSE_train = []

for i in range(folds_index):
    test = df[i * fold_size:(i + 1) * fold_size]
    if i == 0:
        train = df[(i + 1) * fold_size:]
    else:
        train = df[:i * fold_size]
        if i != 4:
            train = train.append(df[(i + 1) * fold_size:], ignore_index=False)

    Features = train.drop('Target', axis=1)
    Target = train['Target']
    w0, w1 = sgdFFM(Features, Target)

    train_pred = predict(Features, w0, w1)

    RMSE_train.append(RMSE(train_pred, train['Target']))

    test_pred = predict(Features, w0, w1)

    RMSE_test.append(RMSE(test_pred, test['Target']))


KeyError: '[ 5311  5051  6420 17568 20939 19769 28693  6396 29419 27480  8666 25658\n 18942 24233 18431  2747 25551 26382   189 31677 19118  3005 21042  1899\n 24118  1267 31551 17912 11394  3556  3890  8838 30740 27464 14502 21777\n 10627  8792 10555 10253  8433 10233 11016 23897  2612 23425 25939 22619\n 21870 23483 26054 15787 27132 17159 12206  8226 14541  3152 26531  1585\n  3943 23939 19457  1021] not in index'